In [2]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import json
import os
import matplotlib.pyplot as plt


In [3]:
import sys
import os
root_path = os.path.abspath("..")
sys.path.append(root_path)

print("Project root added:", root_path)


Project root added: c:\Users\Vishal\Desktop\Collab_Task\collaborative_cnn_team04


In [4]:
from models.model_v2 import CNNModelV2


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNNModelV2()
model.load_state_dict(torch.load("../models/model_v2.pth", map_location=device))
model.to(device)
model.eval()

print("Model V2 loaded!")


Model V2 loaded!


In [6]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

test_data_user1 = datasets.ImageFolder(
    "../data/test",
    transform=test_transform
)

test_loader_user1 = DataLoader(
    test_data_user1,
    batch_size=32,
    shuffle=False
)

len(test_data_user1)


2023

In [7]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader_user1:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        correct += (predicted == labels).sum().item()
        total += labels.size(0)

v2_user1_accuracy = correct / total
v2_user1_accuracy


0.8423133959466139

In [9]:
import json
import os

results = {
    "model": "model_v2",
    "tested_on": "User1 dataset",
    "accuracy": v2_user1_accuracy,
    "total_samples": total
}

os.makedirs("../results", exist_ok=True)

with open("../results/test_v2_user1.json", "w") as f:
    json.dump(results, f, indent=4)

print("Saved to results/test_v2_user1.json")


Saved to results/test_v2_user1.json


In [12]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

test_data_user1 = datasets.ImageFolder("../data/test", transform=test_transform)
test_loader = DataLoader(test_data_user1, batch_size=16, shuffle=False)

print("Loaded User1 Test Dataset")
print("Total Images:", len(test_data_user1))
print("Classes:", test_data_user1.classes)


Loaded User1 Test Dataset
Total Images: 2023
Classes: ['cats', 'dogs']


In [13]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import os

model.eval()

def get_gradcam_v2(model, x, target_layer, class_idx=None):
    conv_out = []
    grads = []

    def fwd_hook(module, inp, out):
        conv_out.append(out)

    def bwd_hook(module, grad_in, grad_out):
        grads.append(grad_out[0])

    h1 = target_layer.register_forward_hook(fwd_hook)
    h2 = target_layer.register_backward_hook(bwd_hook)

    x = x.to(device)
    out = model(x)
    if class_idx is None:
        class_idx = out.argmax(dim=1).item()

    score = out[:, class_idx]
    model.zero_grad()
    score.backward()

    
    h1.remove()
    h2.remove()

    
    activations = conv_out[0]          
    gradients = grads[0]               

    weights = gradients.mean(dim=(2, 3), keepdim=True)  
    cam = (weights * activations).sum(dim=1, keepdim=True)  
    cam = F.relu(cam)


    cam = F.interpolate(
        cam,
        size=(x.size(2), x.size(3)),
        mode="bilinear",
        align_corners=False
    )

    cam = cam[0, 0].detach().cpu().numpy()
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
    return cam


In [14]:
save_dir = "../results/gradcam"
os.makedirs(save_dir, exist_ok=True)

classes = ["cat", "dog"]   # 0 = cat, 1 = dog

num_samples = 5
saved = 0

model.eval()

for images, labels in test_loader:
    batch_size = images.size(0)
    for i in range(batch_size):
        if saved >= num_samples:
            break

        img = images[i].unsqueeze(0)  
        label = labels[i].item()

        cam = get_gradcam_v2(model, img, model.conv3)

        
        img_np = images[i].permute(1, 2, 0).detach().cpu().numpy()
        img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min() + 1e-8)

        
        heatmap = plt.cm.jet(cam)[..., :3]
        overlay = 0.5 * img_np + 0.5 * heatmap
        overlay = np.clip(overlay, 0, 1)

        true_name = classes[label]

        plt.figure(figsize=(6, 3))
        plt.subplot(1, 2, 1)
        plt.imshow(img_np)
        plt.axis("off")
        plt.title(f"Original\nTrue: {true_name}")

        plt.subplot(1, 2, 2)
        plt.imshow(overlay)
        plt.axis("off")
        plt.title("Grad-CAM (Model V2)")

        fname = os.path.join(save_dir, f"v2_sample_{saved}.png")
        plt.tight_layout()
        plt.savefig(fname, dpi=300)
        plt.close()

        saved += 1

    if saved >= num_samples:
        break

print(f"Saved {saved} Grad-CAM images for Model V2 in {save_dir}")


c:\Users\Vishal\Desktop\Collab_Task\collaborative_cnn_team04\venv\Lib\site-packages\torch\nn\modules\module.py:1866: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Saved 5 Grad-CAM images for Model V2 in ../results/gradcam
